# Machine Learning - Assignment 1

> Hoang Dang - s3927234

<!-- ## Abstract -->

<!-- In this assignment,  -->

## Problem Formulation

- Supervised
- Multivariate logistic regression
- Binary classification problem

> Xem lại tut w2 và w3 để tham khảo

## Exploratory Data Analysis (EDA)

The following describes the dataset columns:

| Attribute             | Description                                                                                                              |
|-----------------------|--------------------------------------------------------------------------------------------------------------------------|
| ID                    | Patient ID                                                                                                               |
| Status                | 0 = no diabetes, 1 = prediabetes or diabetes                                                                             |
| HighBP                | 0 = no high blood pressure, 1 = high blood pressure                                                                      |
| HighChol              | 0 = no high cholesterol, 1 = high cholesterol                                                                            |
| CholCheck             | 0 = no cholesterol check in 5 years, 1 = yes cholesterol check in 5 years                                                |
| BMI                   | Body Mass Index                                                                                                          |
| Smoker                | Have you smoked at least 100 cigarettes in your entire life? (0 = no, 1 = yes)                                           |
| Stroke                | (Ever told) you had a stroke (0 = no, 1 = yes)                                                                          |
| HeartDiseaseorAttack | Coronary heart disease (CHD) or myocardial infarction (MI) (0 = no, 1 = yes)                                             |
| PhysActivity          | Physical activity in the past 30 days (not including job) (0 = no, 1 = yes)                                               |
| Fruits                | Consume fruit 1 or more times per day (0 = no, 1 = yes)                                                                 |
| Veggies               | Consume vegetables 1 or more times per day (0 = no, 1 = yes)                                                            |
| HvyAlcoholConsump     | Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week) (0 = no, 1 = yes) |
| AnyHealthcare         | Have any kind of health care coverage, including health insurance, prepaid plans such as HMO, etc. (0 = no, 1 = yes)   |
| NoDocbcCost           | Was there a time in the past 12 months when you needed to see a doctor but could not because of cost? (0 = no, 1 = yes)|
| GenHlth               | Would you say that in general your health is (scale 1-5): 1 = excellent, 2 = very good, 3 = good, 4 = fair, 5 = poor     |
| MentHlth              | Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good? (scale 1-30 days) |
| PhysHlth              | Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good? (scale 1-30 days) |
| DiffWalk              | Do you have serious difficulty walking or climbing stairs? (0 = no, 1 = yes)                                            |
| Sex                   | Sex (0 = female, 1 = male)                                                                                              |
| Age                   | 13-level age category: 1 = 18-24, 9 = 60-64, 13 = 80 or older                                                            |
| Education             | Education level (scale 1-6): 1 = Never attended school or only kindergarten, 2 = Grades 1 through 8 (Elementary), 3 = Grades 9 through 11 (Some high school), 4 = Grade 12 or GED (High school graduate), 5 = College 1 year to 3 years (Some college or technical school), 6 = College 4 years or more (College graduate) |
| Income                | Income scale (scale 1-8): 1 = less than $10,000, 5 = less than $35,000, 8 = $75,000 or more                             |
| ExtraMedTest          | The result of an extra medical test, range (-100, 100)                                                                  |
| ExtraAlcoholTest      | The result of an extra alcohol test, range (-100, 100)                                                                  |


- The target variable is "Status," which indicates the presence or absence of diabetes. It's a binary classification problem (0 = no diabetes, 1 = prediabetes or diabetes).

---

- Perform thorough exploratory data analysis to understand the distribution of features, identify correlations, and visualize patterns in the dataset

- Handle missing values, outliers, and perform necessary data preprocessing steps


https://www.itl.nist.gov/div898/handbook/eda/section1/eda11.htm


- Có cần check duplicated data?



## Model Proposal

- Propose at least three different machine learning models suitable for predicting the status of diabetes development in patients (only use techniques taught in class up to week 5 - inclusive)

- Justify the selection of each model based on its strengths, weaknesses, and suitability for the task


## Model Implementation

- Implement the proposed machine learning models using appropriate libraries (e.g scikit-learn, TensorFlow, Keras, etc.).

- Fine-tune hyperparameters using cross-validation techniques to optimize model performance


## Model Evaluataion

Submit the prediction for patients in a given test set whose dependent variable (i.e., the presence or absence of diabetes) is hidden


## Discussion and Conclusion

- Provide a detailed discussion on the effectiveness of different machine learning models for diabetes prediction.

- Discuss the implications of the findings and potential applications of the predictive models in healthcare settings.

- Highlight areas for future research or improvements in predictive modeling for diabetes